# 爬虫学习笔记

> ref： 阿里云大学 - Python网络爬虫快速入门到精通 - 韦玮

__Content__
* urllib
* requests
* scrapy

## urllib

### 基本操作

In [1]:
import urllib
import urllib.request
import re

data = urllib.request.urlopen('http://www.jd.com').read().decode('UTF-8', 'ignore')
data

'<!DOCTYPE html>\n<html>\n\n<head>\n    <meta charset="utf8" version=\'1\'/>\n    <title>京东(JD.COM)-正品低价、品质保障、配送及时、轻松购物！</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=yes"/>\n    <meta name="description"\n          content="京东JD.COM-专业的综合网上购物商城,销售家电、数码通讯、电脑、家居百货、服装服饰、母婴、图书、食品等数万个品牌优质商品.便捷、诚信的服务，为您提供愉悦的网上购物体验!"/>\n    <meta name="Keywords" content="网上购物,网上商城,手机,笔记本,电脑,MP3,CD,VCD,DV,相机,数码,配件,手表,存储卡,京东"/>\n    <script type="text/javascript">\n        window.point = {}\n        window.point.start = new Date().getTime()\n    </script>\n    <link rel="dns-prefetch" href="//static.360buyimg.com"/>\n    <link rel="dns-prefetch" href="//misc.360buyimg.com"/>\n    <link rel="dns-prefetch" href="//img10.360buyimg.com"/>\n    <link rel="dns-prefetch" href="//img11.360buyimg.com"/>\n    <link rel="dns-prefetch" href="//img12.360buyimg.com"/>\n    <link rel="dns-prefetch" href="//img13.360buyimg.com"/>\n    <link rel="dns-prefetch

In [2]:
# Get title
pat = '<title>(.*?)</title>'
re.compile(pat, re.S).findall(data)

['京东(JD.COM)-正品低价、品质保障、配送及时、轻松购物！']

In [7]:
# Save to Disk
urllib.request.urlretrieve('http://www.jd.com', filename='./tmp/jd.html')

('./tmp/jd.html', <http.client.HTTPMessage at 0x1049d7c10>)

### 浏览器伪装

In [4]:
opener = urllib.request.build_opener()
UA = ('User-Agent', 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11')
opener.addheaders = [UA]
urllib.request.install_opener(opener)
data = urllib.request.urlopen('https://www.qiushibaike.com').read().decode('UTF-8','ignore')
data

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf8">\r\n<title>友际无限（北京）科技有限公司</title>\r\n<meta name="Keywords" content="友际无限（北京）科技有限公司">\r\n<meta name="Description" content="友际无限（北京）科技有限公司">\r\n\r\n<link rel="stylesheet" type="text/css" href="E6bmVsdm\\template\\css\\style.css">\r\n<!--[if lt IE 9]>\r\n<script src="/Public/js/html5shiv.js"></script>\r\n<![endif]-->\r\n\r\n<!--[if lt IE 9]>\r\n<script type="text/javascript" src="/template/js/jquery-1.10.2.min.js"></script>\r\n<![endif]-->\r\n<!--[if gte IE 9]><!-->\r\n<script type="text/javascript" src="E6bmVsdm\\template\\js\\jquery-2.2.3.min.js"></script>\r\n<script type="text/javascript" src="E6bmVsdm\\template\\js\\layer.js"></script>\r\n<!--<![endif]-->\r\n\r\n\r\n<script type="text/javascript" src="E6bmVsdm\\template\\js\\self.js"></scri

### 构建用户代理池

In [5]:
import random

url = 'https://www.qiushibaike.com'
uapools = [
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36'
]

def UA():
    opener = urllib.request.build_opener()
    thisua = random.choice(uapools)
    ua = ('User-Agent', thisua)
    opener.addheaders = [ua]
    urllib.request.install_opener(opener)
    print('当前使用UA：', str(thisua))
    
for i in range(0, 10):
    if(i%3==0):
        UA()
    data = urllib.request.urlopen(url).read().decode('UTF-8', 'ignore')
    print(len(data))



当前使用UA： Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2
9869
9869
9869
当前使用UA： Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2
9869
9869
9869
当前使用UA： Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60
9869
9869
9869
当前使用UA： Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0
9869


### Example - 抓取我的项目portfolio并分页

In [10]:
import random

url = 'https://www.qiushibaike.com'
uapools = [
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36'
]

def UA():
    opener = urllib.request.build_opener()
    thisua = random.choice(uapools)
    ua = ('User-Agent', thisua)
    opener.addheaders = [ua]
    urllib.request.install_opener(opener)
    print('当前使用UA：', str(thisua))
    
for i in range(0, 4):
    UA()
    if i==0:
        thisurl = 'https://www.aemon.wang'
    else:    
        thisurl = 'https://www.aemon.wang/blog/page'+str(i+1)+'/'
    data = urllib.request.urlopen(thisurl).read().decode('UTF-8', 'ignore')
    pat = '<div class="excerpt">(.*?)</div>'
    rst = re.compile(pat, re.S).findall(data)
    for j in range(0, len(rst)):
        print(rst[j])
        print('-----------')

当前使用UA： Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60

              I am so happy finaly I get something could show on Showcase, maybe not perfect, but still is my half year working. I am appriciated I am a part of BIT project also in MobileGroup. I would say thank you to my lecturer Groyyson, my all mates here! There...
          
-----------

              Tomorrow will be our last day this semester, also it is Showcase day which one of the very important BIT activity. Eventhough We haven’t absluotely meet all the requirements from our client. I think that I still need tell client what we have done though? So I need send...
          
-----------

              I felt bored last night just want to experience the latest Windows 10 terminal, and then updated the system by the way, and also updated the nodejs, and found that I could not start my project today. I really think that I am very bad, the last wee


              In the first Sprint, I and Carthor each assigned 2 tasks, I was responsible for the user Sign up and login page, and created a database account, the request is based on document type data such as Firebase or MongoDB, I chose Firebase.


          
-----------

              Today is the first debreif, the debreif is a new word for me. Also for this paper, all the experiences are totally new. I didn’t expect it to be four weeks so soon, and our entire semester was less than 16 weeks. Recalling these four weeks, what did I...
          
-----------

              The first Sprint will end this week, and I have a pretty one hour meeting with Carthur. We summed up the work tasks of the previous week and how to implement them, and so on. Each of us also exchanged ideas with. He showed me the results of his...
          
-----------

              There are many werid circumstances when I develop that code, it is not about official documents or principles could handle it, just

### 抓包 - Fiddler & Firefox

__Configuration__

1. Firefox - 选项 - 网络设置 - 手动http代理： 127.0.0.1:8888 勾选https
2. Fiddler - Tools - Option - Https - 勾选所有选项, Actions - Export Root Certificate to Desktop
3. Firfox - 选项 - 证书 - 查看证书 - 证书颁发机构 - 导入证书

### Example - 抓包分析腾讯视频评论

In [ ]:
import urllib.request
import re

cid = '6629768780947370057'

for i in range(0,100):
    print('Page:', str(i+1))
    print(' ')
    url = 'https://video.coral.qq.com/varticle/4748040610/comment/v2?callback=_varticle4748040610commentv2&orinum=10&oriorder=o&pageflag=1&cursor='+ cid +'&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=132&_=1582411536658'
    data = urllib.request.urlopen(url).read().decode('UTF-8', 'ignore')
    pat1 = '"content":"(.*?)"'
    comments = re.compile(pat1, re.S).findall(data)
    for item in comments:
        print(str(item))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    pat2 = '"last":"(.*?)"'
    cid = re.compile(pat2, re.S).findall(data)[0] # 从爬下的内容里找下一条的ID

## requests

__Basis__

* 请求方式：get/post/put/delete
* 参数：params/headers/proxies/cookies/data
* text 响应数据
* content 相应数据（二进制）
* encoding 网页编码
* cookies 响应cookie
* url 当前请求url
* status_code 状态码

In [ ]:
import requests
import re

userPool = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36'}
px = {'http':'http://127.0.0.1:8888'}
rst = requests.get('http://www.aliwx.com.cn/', headers = userPool)
title = re.compile('<title>(.*?)</title>', re.S).findall(rst.text)
title

In [ ]:
rst.status_code

In [ ]:
rst.text

In [ ]:
rst.content

In [ ]:
rst.encoding

In [ ]:
rst.cookies

In [ ]:
requests.utils.dict_from_cookiejar(rst.cookies)

In [ ]:
pr = {'wd': '阿里文学',}
rst = requests.get('https://www.baidu.com/s', params = pr)
rst.url

### Example - 爬云栖社区博文

In [9]:
import requests
import re
import time
import random

key = 'Python'
url = 'https://yq.aliyun.com/search/articles/'
data = requests.get(url,params={'q': key}).text
pat1 = '<div class="_search-info">找到(.*?)条关于'
articles_number = re.compile(pat1, re.S).findall(data)[0]
page_number = int(articles_number) // 15 + 1

for i in range(0, 5): #int(page_number) 这里我只测试5页，毕竟数据太多
    print('--- 正在爬第'+ str(i + 1) +'页--- ')
    index = str(i + 1)
    getdata = { "q": key, "p": index}
    subData = requests.get(url, params=getdata).text
    pat_url = '<div class="media-body text-overflow">.*?<a href="(.*?)">'
    articles = re.compile(pat_url, re.S).findall(data)
    for j in articles:
        thisurl = 'https://yq.aliyun.com' + j
        thisdata = requests.get(thisurl).text
        pat_title = '<p class="hiddenTitle">(.*?)</p>'
        pat_content = '<div class="content-detail unsafe markdown-body">(.*?)</div>'
        title = re.compile(pat_title, re.S).findall(thisdata)[0]
        content = re.compile(pat_content, re.S).findall(thisdata)[0]
        fh = open('./tmp/aliyun_blogs/' + str(i) + '_' + str(time.time()) + '.md', 'w', encoding = 'UTF-8')
        fh.write(title + "<br /><br />" + content)
        fh.close()
        

--- 正在爬第1页--- 
--- 正在爬第2页--- 
--- 正在爬第3页--- 
--- 正在爬第4页--- 
--- 正在爬第5页--- 


## Scrapy

### 常用命令

```bash

brew install scrapy # installation

scrapy startproject project_name # create a new project

# basic project structure
.
├── firstsp
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

scrapy genspider -l # check how many templates does there have

scrapy genspider -t basic fst aliwx.com.cn # create a spider file with basic template, fst as name and domain under the spiders folder

scrapy crawl fst # run a spider with name fst

scrapy list # list how many spiders at the moment



```

### 基本流程
1. 创建项目

2. 定义items # 创建字段

```python
import scrapy


class FirstspItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
```

3. 创建spider文件

```python
# -*- coding: utf-8 -*-
import scrapy
from firstsp.items import FirstspItem // It comes form item file

class FstSpider(scrapy.Spider):
    name = 'fst'
    allowed_domains = ['aliwx.com.cn']
    start_urls = ['http://www.aliwx.com.cn/']

    def parse(self, response):
        item = FirstspItem()
        item['title'] = response.xpath("//p[@class='title']/text()").extract()
        yield item # to pipelines
```

4. 编写pipelines

```python
class FirstspPipeline(object):
    def process_item(self, item, spider):
        for i in range(0, len(item['title'])):
            print('----------')
            print(item['title'][i])
        return item
```

5. 配置setting, uncomment the code below and change name 'FirstspPipeline'

```python
ITEM_PIPELINES = {
   'firstsp.pipelines.FirstspPipeline': 300,
}
```

__Example code__

___See local project___

### Example - 51job

In [ ]:
import re
import requests

key = 'Python'
data = {
    "fromJs": "1",
    "jobarea": "020000",
    "keyword": key,
    "keywordtype": "2",
    "curr_page": "1",
}
hd = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36",
    #"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0",
    #"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2",
    #"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
}
response = requests.get("http://search.51job.com/jobsearch/search_result.php", params=data, headers=hd)
data1 = bytes(response.text, response.encoding).decode("gbk", "ignore")
pat_page = "共(.*?)条职位"
allline = re.compile(pat_page, re.S).findall(data1)[0]
print('总数量：', allline)
allpage = int(allline)//50 + 1
for i in range(0, 3): # here should be allpage if you wanna grab all pages
    print("-----正在爬" + str(i + 1) + "页-----")
    getdata = {
        "fromJs": "1",
        "jobarea": "020000",
        "keyword": key,
        "keywordtype": "2",
        "curr_page": str(i + 1),
    }
    response = requests.get("http://search.51job.com/jobsearch/search_result.php", params=getdata, headers=hd)
    thisdata = bytes(response.text, response.encoding).decode("gbk", "ignore")
    job_url_pat = 'name="delivery_jobid".*?href="https://jobs.51job.com/(.*?).html'
    job_url_all = re.compile(job_url_pat, re.S).findall(thisdata)
    print('共有信息：', len(job_url_all))
    for job_url in job_url_all:
        thisurl = "http://jobs.51job.com/" + job_url + ".html"
        response = requests.get(thisurl)
        thisdata = bytes(response.text, response.encoding).decode("gbk", "ignore")
        pat_title = '<h1 title="(.*?)"'
        pat_company = '<p class="cname">' + '.*?title="(.*?)"'
        pat_money = '<div class="tHeader tHjob">.*?<strong>(.*?)</strong>'
        pat_msg = '<div class="bmsg job_msg inbox">(.*?)<div class="share">'
        title = re.compile(pat_title, re.S).findall(thisdata)[0]
        company = re.compile(pat_company, re.S).findall(thisdata)[0]
        money = re.compile(pat_money, re.S).findall(thisdata)[0]
#         msg = re.compile(pat_msg, re.S).findall(thisdata)[0]
        print("----------")
        print(title)
        print(company)
        print(money)
#         print(msg)

### Example - 淘宝网商品信息

__分析__

```javascript
第一页
https://s.taobao.com/list?q=%E5%A5%B3%E8%A3%85&cat=16&style=grid&seller_type=taobao&spm=a217f.8051907.1000187.1

第二页
https://s.taobao.com/list?q=%E5%A5%B3%E8%A3%85&cat=16&style=grid&seller_type=taobao&spm=a217f.8051907.1000187.1&bcoffset=0&s=60

第三页
https://s.taobao.com/list?q=%E5%A5%B3%E8%A3%85&cat=16&style=grid&seller_type=taobao&spm=a217f.8051907.1000187.1&bcoffset=0&s=120

页码规律
S = 60*（page-1）

标题
"raw_title":"2020春装新款大码女装胖mm宽松显瘦西装外套洋气阔腿裤套装200斤"

价格
"view_price":"228.00"

详细页链接
<a id="J_Itemlist_TLink_610721755943" class="J_ClickStat" data-nid="610721755943" href="//item.taobao.com/item.htm?id=610721755943&amp;ns=1&amp;abbucket=18" target="_blank" trace="lsrp_auction" traceidx="41" trace-index="41" trace-nid="610721755943" trace-num="60" trace-price="178.00" trace-pid="777289949" data-spm-anchor-id="a219r.lm874.14.367">
<span class="baoyou-intitle icon-service-free"></span>
法儿家2020新款<span class="H">女装</span>流行春秋网纱百褶中长款粉色仙女裙油画半身裙
</a>

评论数（抓包获取）
https://rate.taobao.com/detailCount.do?_ksTS=1582613399228_117&callback=jsonp118&itemId=610721755943
    
```

In [ ]:
import urllib.request
import re
import random

keyname = "女装"
key = urllib.request.quote(keyname)
uapools =[
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36'
]

def ua(uapools):
    thisua = random.choice(uapools)
    print(thisua)
    headers = ("User-Agent", thisua)
    opener = urllib.request.build_opener()
    opener.addheaders = [headers]
    urllib.request.install_opener(opener) # 安装为全局
    
for i in range(1, 101):
    print("-----第" + str(i) + "页商品-----")
    url = "https://s.taobao.com/list?q="+ key +"&cat=16&style=grid&seller_type=taobao&s="+ str((i-1)*60)
    ua(uapools)
    data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    pat = '"nid":"(.*?)"'
    idlist = re.compile(pat).findall(data)
    for j in range(0, len(idlist)):
        thisid = idlist[j]
        thisurl = "https://item.taobao.com/item.htm?id=" + str(thisid)
        itemdata = urllib.request.urlopen(thisurl).read().decode("utf-8", "ignore")
        titlepat = ''
        detailpat = ''
        pricepat = ''
        title = re.compile(titlepat, re.S).findall(itemdata)
        if len(title)>0:
            title = title[0]
        else:
            continue
        if len(price)>0:
            price = price[0]
        else:
            price = 0
        print('info')
            
    

### Example - 抓取Trademe 奥塔哥但尼丁房子信息

In [ ]:
import re
import requests

html_pattern = re.compile(r'<[^>]+>',re.S) # filter html tags
key_word = 'dunedin%2c+otago'
cid = 3399
key = 1473476618
data = {
    "searchstring": key_word,
    "cid": cid,
    "key": key,
    "page": "1",
}
hd = {
    #"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0",
    #"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2",
    #"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
}
response = requests.get("https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx", params=data, headers=hd)
data1 = bytes(response.text, response.encoding).decode("gbk", "ignore")
pat_page = '"originalListingCountText":"(.*?) listings, showing'
allline = re.compile(pat_page, re.S).findall(data1)[0]
allpage = int(allline)//24 + 1
print('The spider finds {} listings totally in {} pages'.format(allline, allpage))
fh = open('./trademe.csv', 'w', encoding = 'UTF-8')
fh.write('title, price, description, address\n')
for i in range(0, allpage):
    print("----- The Spider is working on page " + str(i + 1))
    getdata = {
        "searchstring": key_word,
        "cid": cid,
        "key": key,
        "page": str(i + 1),
    }
    response = requests.get("https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx", params=getdata, headers=hd)
    thisdata = bytes(response.text, response.encoding).decode("UTF-8", "ignore")
    job_url_pat = 'class="tmp-search-card-top-tier__link" href="(.*?)"'
    job_url_all = re.compile(job_url_pat, re.S).findall(thisdata)
    print('----- It finds {} items on page {}'.format(len(job_url_all), str(i+1)))
    for job_url in job_url_all:
        thisurl = "https://www.trademe.co.nz/" + job_url
        response = requests.get(thisurl)
        thisdata = bytes(response.text, response.encoding).decode("UTF-8", "ignore")
        pat_title = '<div class="property-title">(.*?)</h1>'
        pat_price = '<table id="ListingAttributes" class="ListingAttributes">.*?Price:(.*?)</td>'
        pat_description = '<div id="ListingDescription_ListingDescription" class="ListingDescription">(.*?)</div>'
        pat_address = '<table id="ListingAttributes" class="ListingAttributes">.*?Location:(.*?)</td>'
        title = re.compile(pat_title, re.S).findall(thisdata)[0].replace("<h1>", "").strip()
        price = re.compile(pat_price, re.S).findall(thisdata)[0].replace("</th>", "").replace("<td>", "").strip()
        description = re.compile(pat_description, re.S).findall(thisdata)[0]
        description = html_pattern.sub('', description).strip()
        address = re.compile(pat_address, re.S).findall(thisdata)[0].replace("</th>", "").replace("<td>", "").strip()
        print("~~~~~")
        print('Title: ', title)
        print('Price: ', price)
        print('Description: ', description)
        print('Address: ', address)
        fh.write(title + "," + price + "," + description + "," + address + '\n')

fh.close()